# Análisis de Series Temporales
## Laboratorio 1

Estudiante: Ing. Luis Epifanio (luis.epifanio@gmail.com)

# 1. Consigna

1) Levantar los datos desde el archivo dataset/Telefonia_movil.csv. Los mismos fueron extraídos de http://datosabiertos.enacom.gob.ar/visualizations/29890/trafico-de -suscriptores-de-telefonia-movil-miles-de-minutos/ 

> Ver apartado [2.1. Carga y Limpieza de Datos](#2.1.-Carga-y-Limpieza-de-Datos)

2) Explorar los datos, visualizarlos.

> Ver apartado [2.2.-Análisis-Exploratorio](#2.2.-Análisis-Exploratorio.)

3) Implementar dos modelos (como ser Random Walk (Naïve) y Exponential Smoothing).

> Ver apartado [2.3.-Implementación-de-Modelos](#2.3.-Implementación-de-Modelos)
>
> -[2.3.1.-Random-Walk-(Naïve)](#2.3.1.-Random-Walk-(Naïve))
>
> -[2.3.2.-Exponential-Smoothing](#2.3.2.-Exponential-Smoothing)

4) Visualizar los resultados y evaluarlos, comparando ambas implementaciones.

> Ver apartado [4.-Comparación-y-Selección-del-Modelo](#4.-Comparación-y-Selección-del-Modelo)


# 2. Desarrollo

## 2.1. Carga y Limpieza de Datos

Correspondiente al punto 1: Levantar los datos desde el archivo dataset/Telefonia_movil.csv. Los mismos fueron extraídos de http://datosabiertos.enacom.gob.ar/visualizations/29890/trafico-de -suscriptores-de-telefonia-movil-miles-de-minutos/

In [1]:
from plotting import fanchart
from scipy import stats

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm

ModuleNotFoundError: No module named 'plotting'

Algunas funciones y configuracioens de soporte

In [ ]:
# Plot settings
sns.set_context('notebook') 
sns.set_style('ticks')
red='#D62728'
green='#227722'
blue='#1F77B4'
black='#000000'
%matplotlib inline

In [ ]:
plt.rcParams['figure.figsize'] = [15, 10]
plt.style.use('ggplot')

In [ ]:
def asQuarter(year, quarter):
    return str(year) + 'Q' + str(quarter)

In [ ]:
# Load
data = pd.read_csv('datasets/Telefonia_movil.csv', parse_dates=True, dayfirst=True, thousands='.')
#Create Quarter column as new index to be
data['quarter'] = data.apply(lambda x: asQuarter(x['Año'], x['Trimestre']), axis=1)
data['quarter'] = pd.to_datetime(data.quarter).dt.to_period('Q')
data=data.set_index('quarter')
#Drop some columns not needed
if ({'Año', 'Trimestre','Periodo'}.issubset(data.columns)):
    data.drop(['Año', 'Trimestre', 'Periodo'], axis=1, inplace=True)
#Rename columns for simplicity
data.rename(columns={
    'Minutos pospago salientes (miles)' : 'pospago', 
    'Minutos prepago salientes (miles)' : 'prepago', 
    'Total de minutos salientes (miles)': 'total'

}, inplace=True)
#TODO use PeriodIndex to build index
data.head(5)

In [ ]:
display(data.dtypes)

In [ ]:
data.isnull().sum(axis = 0)

## 2.2. Análisis Exploratorio.

Correspondiente al punto 2: Explorar los datos, visualizarlos.

In [ ]:
data.tail()

In [ ]:
data.plot()

La serie que tomaremos como foco de analisis será la correspondiente a `Minutos pospago salientes (miles)` o en nuesto dataset `'pospago'`

In [ ]:
serie_name = 'pospago'
y=data[serie_name]

In [ ]:
fig, ax= plt.subplots()
y.plot(color=green)
ax.set_xlabel('')
ax.set_ylabel(serie_name)
ax.set_title(serie_name)
ax.set_xticks([], minor=True) 
sns.despine()
plt.show()

In [ ]:
y.describe().round(2)

## 2.3. Implementación de Modelos

Correspondiente al punto 3: Implementar dos modelos (como ser Random Walk (Naïve) y Exponential Smoothing).

### 2.3.1. Random Walk (Naïve)

In [ ]:
def analyzeAndPlotRandowWalk(serie):
    h = 4
    test = pd.period_range(
        start=serie.index[-1]+1, 
        periods=h, 
        freq='Q'
    )
    pred = pd.Series(np.repeat(serie.iloc[-1], h), index=test) # the forecast repeats the last observed values h times
    
    print('Predictions:')
    display(pred)
    
    resid=serie-serie.shift(1) # the shift lags the series by one period
    sigma = resid.std()
    round(sigma,3)

    intv = pd.concat([pred-stats.norm.ppf(0.975)*sigma*np.sqrt(np.arange(1,h+1)),
                       pred+stats.norm.ppf(0.975)*sigma*np.sqrt(np.arange(1,h+1))], axis=1)
    
    print('')
    display(intv.round(3))
    
    point_pred = (intv[0]+intv[1])/2

    fig, ax = fanchart(serie['2013Q1':],point_pred,intv,intv,intv)

    plt.show()

In [ ]:
serie_name = 'pospago'
y = data[serie_name]
analyzeAndPlotRandowWalk(y)

In [ ]:
naive = y.copy()
naive = naive.shift(1)

In [ ]:
fig, ax= plt.subplots(figsize=(15,10))
y.plot(color=red, label='Observado')
naive.plot(color=blue, label='Naïve')
ax.set_xlabel('')
ax.set_ylabel(serie_name)
ax.set_title(serie_name)
ax.set_xticks([], minor=True) 
plt.legend(loc='best')
sns.despine()
plt.show()

### 2.3.2. Exponential Smoothing

#### 2.3.2.1. Modelo y Ajuste con Exponential Smoothing

In [ ]:
def analyzeAndPlotExpSmoothing(y):
    model_ses = sm.tsa.SimpleExpSmoothing(y).fit()
    fitted_ses = model_ses.fittedvalues
    # model_ses.summary() # summary isn't implemented in 0.9, looks it'll be avaible in 0.10
    alpha_ses = model_ses.model.params['smoothing_level']
    mse_ses = sm.tools.eval_measures.mse(y,fitted_ses)
    # loglik_ses = model_ses.model.loglike(model_ses.model.params) # NotImplementedError
    aic_ses = model_ses.aic
    bic_ses = model_ses.bic
    print(' Smoothing parameter: alpha  {0:.3f}'.format(alpha_ses))
    print(' In-sample fit:')
    print(' MSE               {0:.3f}'.format(mse_ses))
    print(' AIC               {0:.3f}'.format(aic_ses))
    print(' BIC               {0:.3f}'.format(bic_ses))
    print()
    N = len(y)
    loglik_ses=-(N/2)*(1+np.log(2*np.pi))-(N/2)*np.log(mse_ses)
    aic_ses=-2*loglik_ses+2*3
    bic_ses=-2*loglik_ses+np.log(N)*3
    print(' Log-likelihood    %.3f' % (loglik_ses))
    print(' AIC               {0:.3f}'.format(aic_ses))
    print(' BIC               {0:.3f}'.format(bic_ses))
    
    fig, ax= plt.subplots(figsize=(15,10))
    y.plot(color=red, label='Inflation')
    fitted_ses.plot(color='blue', label='Simple exponential smoothing fit', alpha=0.8)
    ax.set_xlabel('')
    ax.set_ylabel('Original')
    ax.set_title('Comparison')
    ax.set_xticks([], minor=True) 
    plt.legend(loc='best')
    sns.despine()
    plt.show()
    
    return fitted_ses


In [ ]:
fitted_ses = analyzeAndPlotExpSmoothing(data['pospago'])

#### 2.3.2.2. Diagnóstico del Modelo Generado con Exponential Smoothing

In [ ]:
resid=y-fitted_ses 

In [ ]:
fig, ax= plt.subplots()
resid.plot(color=blue)
ax.set_xlabel('')
ax.set_xticks([], minor=True) 
ax.set_title('Residual plot')
sns.despine()
plt.show()

In [ ]:
lags_num = len(resid) - 1
fig, ax = plt.subplots()
sm.graphics.tsa.plot_acf(resid, lags=lags_num, ax=ax)
sns.despine()
plt.show()

In [ ]:
def hist(series):
    fig, ax= plt.subplots()
    sns.distplot(series, ax=ax, hist_kws={'alpha': 0.8, 'edgecolor':'black', 'color': blue},  
                 kde_kws={'color': 'black', 'alpha': 0.7})
    sns.despine()
    return fig, ax

hist(resid)
plt.show()

#### 2.3.2.3. Pronóstico con el Modelo Generado con Exponential Smoothing

In [ ]:
h=12
w=h*10
data = y
model = sm.tsa.SimpleExpSmoothing
pred_index = y.index[-1] + np.array(range(1,h+1))
pred_rolling = pd.Series(0,index=pred_index)
pred_expanding = pd.Series(0,index=pred_index)
orig_fit = None
for i in range(0,h):
    current = pd.concat([data, pred_expanding.head(i)])
    expanding = model(current).fit()
    if i==0:
        orig_fit = expanding.fittedvalues
    rolling = model(current.tail(w)).fit()
    pred_expanding.update(expanding.forecast(1))
    pred_rolling.update(rolling.forecast(1))
ax = data[-w:].plot(color='red' )
pd.concat([orig_fit[-w:],pred_expanding]).plot(ax=ax,color='blue');
pd.concat([orig_fit[-w:],pred_rolling]).plot(ax=ax,color='green');

# 3. Validación de Modelos

In [ ]:
# Real time forecasting - use it as a template - expanding window

validation=y['2016Q1':].index 
start = y.index.get_loc('2016Q1') 

pred1 = []
pred2 = []
actual= []
for i in range(start, len(y)):
    actual.append(y.iloc[i]) # actual value
    pred1.append(y.iloc[i-1]) # random walk forecast
    model = sm.tsa.SimpleExpSmoothing(y.iloc[:i]).fit() 
    pred2.append(model.forecast(1)[0]) # SES forecast

columns=['RW', 'SES', 'Actual']
results_sm = np.vstack([pred1,pred2,actual]).T
results_sm = pd.DataFrame(results_sm, columns=columns, index=validation)

results_sm.tail()

In [ ]:
from scipy import stats

table = pd.DataFrame(0.0, index=results_sm.columns[:-1], columns=['RMSE','SE'])
for i in range(2):
    table.iloc[i,0] = sm.tools.eval_measures.rmse(results_sm.iloc[:,i], results_sm.iloc[:,-1])
    table.iloc[i,1] = (results_sm.iloc[:,i] - results_sm.iloc[:,-1]).sem()
table.round(3)

# 4. Comparación y Selección del Modelo 

De la sección inmediata anterior, podemos advertir que Random Walk (Naïve) **(RM)** tiene menor RMSE de validación acorde a los datos utilizados, lo cual lo convierte en el mas conveninete de entre los comparados.Se advierte además una elevada Desviación estándar y un set de datos acotado, por lo que sería conveniente analizar algunos otros modelos, tal como veremos en el práctico numero dos.

In [ ]:
fig, ax= plt.subplots( )
y.plot(color=red, label='Observado')
naive.plot(color=blue, label='Naïve')
fitted_ses.plot(color='black', label='Simple exponential smoothing fit', alpha=0.8)
ax.set_xlabel('')
ax.set_ylabel(serie_name)
ax.set_title(serie_name)
ax.set_xticks([], minor=True) 
plt.legend(loc='best')
sns.despine()
plt.show()

In [ ]:
resid_naive=y-y.shift(1)

fig, ax= plt.subplots( )
resid.plot(color=black, label='Smoothing Residual')
resid_naive.plot(color=blue, label='Naïve Residual')
ax.set_xlabel('')
ax.set_xticks([], minor=True) 
ax.set_title('Gráfica de Residuos')
plt.legend(loc='best')
sns.despine()
plt.show()